# Profile classification according to the number of layers

Once we finished our approaches, we are going to classify all our profiles according to their number of layers.

In [1]:
#First, some imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotting_functions as plot

#Now we read the data
from read_CTD import read_CTD
data_dirs = ['C:/Users/Pau/Data/TFM/CTD_proc_files_1mbin/']
data=read_CTD(data_dirs)

#And delete the outliers
from out_outliers import out_outliers
for ncast in data.keys():
    for variable in data[ncast].columns:
        out_outliers(data, ncast, variable)

In [2]:
#Code that creates the layers' classification tables
#Import the different functions that correspond to the different approaches that determines the layers of a vertical profiles
from inflexion_points import inflexion_points     #1st approach
from maxmin import maxmin, maxmin_2               #2nd and 3rd approach
#We will smooth the profiles with a lamb=5.000.000 except for the 1st that will be of 50.000.000
lamb=5000000

#1st we create a series that contains the number of layers of each profile
table1 = pd.Series(dtype=np.int64)
table2 = pd.Series(dtype=np.int64)
table3 = pd.Series(dtype=np.int64)
for ncast in data.keys():
    table1[ncast]=inflexion_points(data, ncast, lamb=50000000)    #Series of the 1st approach
    table2[ncast]=maxmin(data, ncast, lamb=lamb)              #Series of the 2nd approach
    table3[ncast]=maxmin_2(data, ncast, lamb=lamb)            #Series of the 3rd approach
    
#Now, we transform the series and create a table that classifies the profiles according to their number of layers
layers1 = pd.Series(index=set(table1.values), dtype=np.int64)
layers2 = pd.Series(index=set(table2.values), dtype=np.int64)
layers3 = pd.Series(index=set(table3.values), dtype=np.int64)

for a, b, c in zip(table1.values, table2.values, table3.values):
    #print(a,b,c)
    layers1[a]=list(table1[table1==a].index)
    layers2[b]=list(table2[table2==b].index)
    layers3[c]=list(table3[table3==c].index)
layers1.index.name = 'Nº of layers (1st approach) - [ncast profile]'
layers2.index.name = 'Nº of layers (2nd approach) - [ncast profile]'
layers3.index.name = 'Nº of layers (3rd approach) - [ncast profile]'

## CLASSIFICATIONS

We created 3 tables, one for each approach. It is important to remark that the results vary significantly in function of how much we smooth the profile. We opted for a significant smooth to observe just the big scale features. To choose how much we smoothed the profile, we used the functions that plot the profiles separated by its layers and decided subjectively which one fit better.


The 3 classifications are similar, especially the second and the third that are almost exact. We could expect it, because when work with very smooth profiles ,the second approach does not detect many layers closer than 50m. Most of the profiles are classified between 3 and 4 layers. Profiles in the first approach range from 2 to 7 layers and in the second and third approaches range from 2 to 6 layers.


In the next section, we will create some maps to observe better those classifications and see if the layer classifications follow any kind of pattern.


### 1st Approach - Inflexion Points

In [3]:
layers1

Nº of layers (1st approach) - [ncast profile]
2                                 [09, 14, 34, 35, 46]
3     [10, 11, 12, 13, 16, 21, 22, 27, 28, 29, 33, 36]
4    [19, 25, 26, 30, 31, 32, 37, 38, 41, 42, 43, 4...
5                                             [17, 40]
6                                 [15, 18, 20, 24, 39]
7                                                 [23]
dtype: object

### 2nd Approach - Relative Maximums and Minimums

In [4]:
layers2

Nº of layers (2nd approach) - [ncast profile]
2                     [09, 10, 11, 13, 14, 17, 21, 22]
3     [12, 15, 16, 19, 28, 29, 33, 34, 35, 36, 45, 46]
4    [26, 27, 30, 31, 32, 37, 38, 39, 40, 41, 42, 4...
5                                     [18, 23, 24, 25]
6                                                 [20]
dtype: object

### 3rd Approach - Relative Maximums and Minimums v2 (no max/mins closer than 50dbar)

In [5]:
layers3

Nº of layers (3rd approach) - [ncast profile]
2                     [09, 10, 11, 13, 14, 17, 21, 22]
3     [12, 15, 16, 19, 28, 29, 33, 34, 35, 36, 45, 46]
4    [24, 26, 27, 30, 31, 32, 37, 38, 39, 40, 41, 4...
5                                         [18, 23, 25]
6                                                 [20]
dtype: object